Thanks for taking your time to complete this take-home task. 

This is where you can show off all your decision science skills through a hands-on project. Together with the dataset, this notebook presents a series of typical problems we face in our job. We expect you to complete the notebook with insightful analysis, elegant code and clear documentation of your findings (perfectly OK to embed within this notebook, just make sure using a distinct color). Please also feel free to keep in the final notebook any code+result where you believe there is value, even if it's not directly addressing the questions below.

We have designed this test with the hope to only take you about 3 hours. We know it’s still quite a commitment of your time and we really appreciate it. That being said, this test is not timed so feel free to spend more time if needed. We’d love to hear your feedback if it actually cost you longer than we intended. It is easy for us to underestimate the required efforts when we know the answers already! Please also tell us if you haven’t had enough fun 😂


## Hints

*   We encourage our decision scientists to use open-source solutions as much as possible instead of re-inventing the wheels. So please feel free to Google for a solution before writing any complicated code. 
*   If you are running short of time but have lots of extra great ideas you'd like to explore, feel free to write down your thoughts in words or pseudo-code instead of code.

## Context

The project we have here is to validate and monitor two credit risk decision models for personal loan underwriting decisions. The information we collected at the underwriting stage of successful applications (i.e. approved and disbursed) were used as the input features of these two models. The two models were trained to predict whether the customers will repay all scheduled payment in the following X months. 

We can split our data into the following three parts:
1.   the **development sample**: all loans applications before 1st August 2019
2.   the **out-of-time monitoring sample**: all loans applications between 1st August 2019 and 1st Jan 2020
3.   the **post-deployment monitoring sample**: all loans applications since 1st Jan 2020


# Part 1. Data processing

Here we have two synthetic datasets of loan customers. 


The file ***monthly_outcome.csv*** contains the repayment *status* at each scheduled payment *date*. The numeric values in column *status* represent how many monthly repayments they were missing. i.e. 0 means they were up to date without any arrear, 3 means they were at that time missing three repayments. Once a customer reaches a status of missing 4 payments, we marked them as 'D' which means it has defaulted. For simplicity let's assume it as a terminal status and all the following records will be 'D' as well. 

The file ***application.csv*** contains
1.   a few variables about the customer we acquired during their loan applications
2.   a binary variable (*is_bad_12m*) of their repayment performance 12 months after disbursal 

**Target definition**: the binary target variable *is_bad_12m* was created by looking at the arrear status of the customers across the 12 dates where the repayment was scheduled. If at any point they had 3 or more repayments missing, this variable would be set to 1, otherwise 0. 

Each loan is represented by an *unique_id* in these two files.



## Task 1.1: data cleaning

Although we are proud of our data infrastructure, it's always a good practice to sense check and clean up a bit. We have left you a few surprises here. Have fun!



In [2]:
import pandas as pd
import numpy as np

outcomes_df = pd.read_csv('monthly_outcome.csv')
application_df = pd.read_csv('application.csv')


### 1.1.1 Clearning Outcomes

In [4]:
outcomes_df.head(2)

,unique_id,date,status
0,39489,2018-02-02,0
1,65413,2018-02-02,0


In [14]:
# Status contains expected values although some formatting differences in values eg. 0.0 vs 0
outcomes_df.status.value_counts()

status
0.0    470201
0      324703
D       16074
1.0     15524
1       10592
2.0      5259
2        3141
3.0      1296
3        1190
Name: count, dtype: int64

In [27]:
# Make all numerical values in status integers eg. force 0.0 to 0
outcomes_df.status = outcomes_df.status.map(lambda x: x if x=='D' else round(float(x)))

In [30]:
# Check formatting again
outcomes_df.status.value_counts()

status
0    794904
1     26116
D     16074
2      8400
3      2486
Name: count, dtype: int64

In [12]:
# All ids appear 12 times
outcomes_df.unique_id.value_counts()


unique_id
39489    12
31197    12
63209    12
84772    12
27745    12
         ..
10739    12
3603     12
79401    12
83348    12
44277    12
Name: count, Length: 70665, dtype: int64

In [15]:
outcomes_df.date.value_counts()

date
2020-03-23    1602
2020-03-15    1593
2020-04-11    1583
2020-04-01    1563
2020-03-22    1563
              ... 
2018-02-08       1
2018-02-17       1
2018-02-20       1
2018-02-06       1
2018-02-05       1
Name: count, Length: 1232, dtype: int64

In [18]:
# All dates appear valid dates as no errors raised here
outcomes_df.date = pd.to_datetime(outcomes_df.date)

In [21]:
# No missing values
outcomes_df.isna().any()

unique_id    False
date         False
status       False
dtype: bool

#### Check if arrear status is properly set

In [60]:
# Create a 'fake' status column where D is valued at 4 fo easier comparisons later
outcomes_df['fake_status'] = outcomes_df.status.map(lambda x: 4 if x=='D' else round(float(x)))

In [84]:
# Create new column to look at last status of the loan
outcomes_df = outcomes_df.sort_values(["unique_id", "date"])

outcomes_df["prev_status"] = outcomes_df.groupby("unique_id")["fake_status"].shift(1).fillna(0)

In [85]:
# Check if any occurances of a loan changing from defaulted to non-defaulted later
# Looks like no - which is good!
outcomes_df['valid_prev_status'] = outcomes_df.fake_status >= outcomes_df.prev_status
outcomes_df[(outcomes_df.valid_prev_status == False) & (outcomes_df.prev_status == 4)]

### 1.1.2 Cleaning Application

In [5]:
application_df.head(2)


,unique_id,stress_score,is_bad_12m,model_1,model_2,origination_date,loan_term,loan_amount,age_oldest_account,total_value_of_mortgage,current_utilisation,months_since_2_payments_missed,number_of_credit_searches_last_3_months
0,19550,NaN,1.0,0.064978,0.044780,2019-08-31,24.0,3484.0,68.0,-999997.0,366.0,15.0,0.0
1,37749,NaN,0.0,0.006730,0.002496,2019-03-20,36.0,5510.0,177.0,191842.0,72.0,-999997.0,1.0


In [31]:
application_df.dtypes

unique_id                                    int64
stress_score                               float64
is_bad_12m                                 float64
model_1                                    float64
model_2                                    float64
origination_date                            object
loan_term                                  float64
loan_amount                                float64
age_oldest_account                         float64
total_value_of_mortgage                    float64
current_utilisation                        float64
months_since_2_payments_missed             float64
number_of_credit_searches_last_3_months    float64
dtype: object

In [33]:
# Missing values. Expected some missing values of st
application_df.isna().any()

unique_id                                  False
stress_score                                True
is_bad_12m                                  True
model_1                                    False
model_2                                    False
origination_date                           False
loan_term                                  False
loan_amount                                False
age_oldest_account                         False
total_value_of_mortgage                    False
current_utilisation                        False
months_since_2_payments_missed             False
number_of_credit_searches_last_3_months    False
dtype: bool

## Task 1.2: another target
In order to understand the performance of a model before it's too late, we also want to monitor the repayment behaviours after the first few repayments. 

Could you please create another "early-risk" target *is_bad_3m* which represents whether the customers ever had **2 or more** repayments in arrear at any point of their first three scheduled ones?



# Part 2. Model validation

In this part let's assume we are still at the model development stage and look at the development sample only (see definition at the start). We will skip the model training part here (which is too much fun to finish in 3 hours), and assume that we already trained two candidate models. These are of course probabilistic classification model, which you can find their scores in ***application.csv*** as columns *model_1* and *model_2*. 

We need to compare their performance and decide which one to use in production. The winner model, once deployed, will be used for decisions of

*   Loan approval: the score must be above certain threshold (which can be adjusted during operation) for the application to be approved. 
*   Loss estimate: for each approved loan, we use the model output to predict the probability of default. 
*   Pricing: based on the loss estimate, we decide the interest rate to be charged in order to cover potential losses. 









## Task 2.1: classification power

A common metric used in the credit risk modelling world is the Gini coefficient, which can be linearly mapped to ROCAUC if that's a term you are more familiar with. Could you please compare the Gini's between the two models as a first step? 




An extended question: assuming that classification power is all we care about, what are the other reasons to not pick the model with highest Gini? It's enough to just write down your thoughts. 


## Task 2.2: classification power in segments

As the population of future business might have different distributions from the development sample, we would ideally want the chosen model to be performant in all segments. For simplicity let's stick with univariate segments only.

Could you please compare the Gini's between the two models in the segments of all the variables? Feel free to define the segments as you see appropriate. 



## Task 2.3: accuracy 
As we want to use our model for loss estimates and pricing of each customer, could you please check whether the scores (as probabilistic predictions) are accurate with respect to the actual "bad rates" (i.e. the fraction of *is_bad_12m*=1 among customers of similar scores)

## Task 2.4: calibration

We also want to monitor the early risk indicator *is_bad_3m* in case something really bad happens (e.g. a pandemic). For that we need to calibrate our scores to the probability of such short-term outcome. Could you please create the calibrated scores for the two models and validate them? (Hint: if this is not a topic you are familiar with, scikit-learn has some handy utilities)

# Part 3. Model monitoring

The training and validation of a model is just part of the story. A large part of our work is to understand how our models perform in real life deicisioning and how we adapt to the changing market. In this part we will look into the monitoring sample (see definition at the start).

Now let's assume that we have choosen *model_1* and deployed it to production since 1st Jan 2020. On that day, our decision engine started to use that model, and since then only approved applications with *model_1*<0.05. 


## Task 3.1: model performance

How did the model perform in this monitoring sample, before & after its deployment into production? How does that compare to the expectation from the development sample? 

## Task 3.2: why the changes?

If you observe a difference, what do you think can be the reason(s)? How are you going to verify your hypothesis?

## Task 3.3: new variable

You might have noticed that a new variable ***stress_score*** has become available since late 2019. Can you figure out whether there is additional classification power from this variable over our models? 

If so, how would you incorporate it into our decision model?
